# ASEED - Analiza Zamówień E-commerce w Czasie Rzeczywistym

## System analizy zamówień sklepu online z użyciem Kafka + Spark Structured Streaming

🎯 **Cel projektu**: Zbudowanie systemu analizującego zamówienia e-commerce w czasie rzeczywistym

**Architektura**:
- **Order Simulator** → generuje zamówienia (order_id, product_id, price, timestamp)
- **Apache Kafka** → przesyła dane strumieniowo
- **Spark Structured Streaming** → analizuje top produkty w czasie rzeczywistym  
- **Web Dashboard** → wizualizuje wyniki

**Wymagania spełnione**:
- ✅ Kafka topic z zamówieniami (order_id, product_id, price, timestamp)
- ✅ Spark Structured Streaming do agregacji
- ✅ Praktyczne wzorce ETL i streaming aggregations

---

## 1. Symulacja danych zamówień i wysyłka do Kafki

W tej sekcji zademonstrujemy jak generować przykładowe dane zamówień i wysyłać je do tematu Kafka.

**Schema zamówienia**:
```json
{
  "order_id": "string",
  "product_id": "string", 
  "price": "float",
  "timestamp": "datetime",
  "quantity": "int",
  "category": "string"
}
```

In [ ]:
# Importy i konfiguracja
import json
import time
import random
from datetime import datetime
from kafka import KafkaProducer
import pandas as pd

# Przykładowe produkty (zgodne z ASEED)
PRODUCTS = [
    {'id': 'PROD-001', 'name': 'Smart Watch Premium', 'category': 'Electronics', 'price': 299.99},
    {'id': 'PROD-002', 'name': 'Fashion Jacket', 'category': 'Clothing', 'price': 89.99},
    {'id': 'PROD-003', 'name': 'Python Programming Book', 'category': 'Books', 'price': 45.99},
    {'id': 'PROD-004', 'name': 'Coffee Machine Pro', 'category': 'Home', 'price': 199.99},
    {'id': 'PROD-005', 'name': 'Running Shoes', 'category': 'Sports', 'price': 129.99}
]

def generate_order():
    """Generuje pojedyncze zamówienie zgodnie ze schematem"""
    product = random.choice(PRODUCTS)
    quantity = random.randint(1, 3)
    
    order = {
        'order_id': f'ORD-{random.randint(100000, 999999)}',
        'product_id': product['id'],
        'product_name': product['name'],
        'category': product['category'],
        'price': product['price'],
        'quantity': quantity,
        'timestamp': datetime.now().isoformat()
    }
    
    return order

# Przykład wygenerowanego zamówienia
sample_order = generate_order()
print("Przykład zamówienia:")
print(json.dumps(sample_order, indent=2, ensure_ascii=False))

In [ ]:
# Konfiguracja producenta Kafka (DEMO - bez rzeczywistego uruchomienia)
def create_kafka_producer():
    """Tworzy producenta Kafka do wysyłania zamówień"""
    try:
        producer = KafkaProducer(
            bootstrap_servers=['localhost:9092'],
            value_serializer=lambda x: json.dumps(x, default=str).encode('utf-8'),
            key_serializer=lambda x: x.encode('utf-8') if x else None
        )
        return producer
    except Exception as e:
        print(f"⚠️  Kafka niedostępna: {e}")
        print("💡 Aby uruchomić Kafka, użyj: python3 ../aseed.py start")
        return None

def send_orders_to_kafka(producer, topic='orders', count=10):
    """Wysyła zamówienia do tematu Kafka"""
    if not producer:
        print("🔄 Symulacja wysyłania do Kafka (bez rzeczywistego wysyłania):")
        
    orders_sent = []
    for i in range(count):
        order = generate_order()
        orders_sent.append(order)
        
        if producer:
            # Rzeczywiste wysyłanie
            producer.send(topic, key=order['order_id'], value=order)
            print(f"✅ Wysłano zamówienie {i+1}/{count}: {order['product_name']}")
        else:
            # Symulacja
            print(f"📝 [{i+1}/{count}] {order['order_id']}: {order['product_name']} (${order['price']})")
        
        time.sleep(0.1)  # Opóźnienie między zamówieniami
    
    if producer:
        producer.flush()
        print(f"\n🚀 Wysłano {count} zamówień do tematu '{topic}'")
    
    return orders_sent

# Demonstracja (bez rzeczywistego połączenia)
print("=== DEMONSTRACJA WYSYŁANIA ZAMÓWIEŃ DO KAFKA ===")
demo_producer = create_kafka_producer()
sample_orders = send_orders_to_kafka(demo_producer, count=5)

# Wyświetlenie statystyk
df_orders = pd.DataFrame(sample_orders)
print(f"\n📊 Statystyki wygenerowanych zamówień:")
print(f"- Łączna liczba: {len(df_orders)}")
print(f"- Kategorie: {df_orders['category'].value_counts().to_dict()}")
print(f"- Średnia cena: ${df_orders['price'].mean():.2f}")
print(f"- Łączna wartość: ${(df_orders['price'] * df_orders['quantity']).sum():.2f}")

## 2. Konfiguracja Spark Structured Streaming do odbioru danych z Kafki

Spark Structured Streaming umożliwia przetwarzanie strumieni danych w czasie rzeczywistym z wysoką wydajnością i fault-tolerance.

**Kluczowe koncepcje**:
- **DataFrame API** - deklaratywne API do streamingu
- **Checkpointing** - odporność na awarie  
- **Watermarking** - obsługa spóźnionych danych
- **Trigger** - kontrola częstotliwości przetwarzania

In [ ]:
# Konfiguracja Spark Session (DEMO)
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

def create_spark_session():
    """Tworzy Spark Session z odpowiednią konfiguracją"""
    try:
        spark = SparkSession.builder \
            .appName("ASEED-OrderAnalysis") \
            .config("spark.sql.streaming.checkpointLocation", "./checkpoints") \
            .config("spark.sql.adaptive.enabled", "true") \
            .config("spark.sql.adaptive.coalescePartitions.enabled", "true") \
            .getOrCreate()
        
        spark.sparkContext.setLogLevel("WARN")  # Zmniejszenie ilości logów
        return spark
    except Exception as e:
        print(f"⚠️  Błąd tworzenia Spark Session: {e}")
        print("💡 W środowisku ASEED Spark jest automatycznie skonfigurowany")
        return None

# Schema dla zamówień (zgodna z formatem ASEED)
order_schema = StructType([
    StructField("order_id", StringType(), True),
    StructField("product_id", StringType(), True), 
    StructField("product_name", StringType(), True),
    StructField("category", StringType(), True),
    StructField("price", DoubleType(), True),
    StructField("quantity", IntegerType(), True),
    StructField("timestamp", StringType(), True)
])

print("📋 Schema zamówienia:")
for field in order_schema.fields:
    print(f"  - {field.name}: {field.dataType} (nullable: {field.nullable})")

# Demonstracja konfiguracji (bez rzeczywistego Spark)
print("\n🔧 Konfiguracja Spark Structured Streaming:")
print("- Application: ASEED-OrderAnalysis")
print("- Checkpoint: ./checkpoints") 
print("- Adaptive Query Execution: Enabled")
print("- Log Level: WARN")

In [ ]:
# Konfiguracja odczytu z Kafka 
def create_kafka_stream(spark, kafka_servers="localhost:9092", topic="orders"):
    """Tworzy streaming DataFrame z Kafki"""
    if not spark:
        print("⚠️  Brak Spark Session - pokazujemy kod konfiguracyjny:")
        
    kafka_options = {
        "kafka.bootstrap.servers": kafka_servers,
        "subscribe": topic,
        "startingOffsets": "latest",  # Zaczynamy od najnowszych wiadomości
        "failOnDataLoss": "false"     # Kontynuuj mimo utraty danych
    }
    
    print("📡 Konfiguracja odczytu z Kafka:")
    for key, value in kafka_options.items():
        print(f"  - {key}: {value}")
    
    if not spark:
        return None
        
    # Streaming DataFrame z Kafki
    kafka_df = spark \
        .readStream \
        .format("kafka") \
        .options(**kafka_options) \
        .load()
    
    # Parsing JSON z wartości Kafka
    orders_df = kafka_df.select(
        col("key").cast("string").alias("order_key"),
        from_json(col("value").cast("string"), order_schema).alias("order")
    ).select("order_key", "order.*")
    
    # Konwersja timestamp do właściwego typu
    orders_df = orders_df.withColumn(
        "timestamp", 
        to_timestamp(col("timestamp"), "yyyy-MM-dd'T'HH:mm:ss.SSSSSS")
    )
    
    return orders_df

# Demonstracja (bez rzeczywistego Spark)
print("=== KONFIGURACJA STREAMING DATAFRAME ===")
demo_stream = create_kafka_stream(None)

print("\n🔄 Logiczny plan odczytu z Kafki:")
print("1. Połączenie z Kafka (localhost:9092)")
print("2. Subskrybcja tematu 'orders'") 
print("3. Parsing JSON z kolumny 'value'")
print("4. Konwersja timestamp do właściwego typu")
print("5. Utworzenie streaming DataFrame z kolumnami:")
print("   - order_key, order_id, product_id, product_name")
print("   - category, price, quantity, timestamp")

## 3. Agregacje strumieniowe: najpopularniejsze produkty

**Streaming Aggregations** to kluczowa funkcja Spark Structured Streaming pozwalająca na:
- Obliczanie metryk w czasie rzeczywistym
- Okna czasowe (tumbling, sliding, session windows)
- Watermarking dla spóźnionych danych
- Stateful operations z checkpointingiem

**Przypadki użycia w ASEED**:
- Top produkty według liczby zamówień
- Przychody według kategorii
- Trendy sprzedaży w oknie czasowym

In [ ]:
# Funkcje agregacji dla top produktów
def create_top_products_aggregation(orders_df):
    """Tworzy agregację top produktów w czasie rzeczywistym"""
    if not orders_df:
        print("⚠️  Brak streaming DataFrame - pokazujemy logikę agregacji:")
        print("""
        # Top produkty - agregacja grupowa  
        top_products = orders_df \\
            .withWatermark("timestamp", "10 minutes") \\
            .groupBy("product_id", "product_name", "category") \\
            .agg(
                count("*").alias("order_count"),
                sum("quantity").alias("total_quantity"), 
                sum(col("price") * col("quantity")).alias("total_revenue"),
                max("timestamp").alias("last_order_time")
            ) \\
            .orderBy(desc("order_count"))
        """)
        return None
        
    # Rzeczywista agregacja (gdyby był DataFrame)
    top_products = orders_df \
        .withWatermark("timestamp", "10 minutes") \
        .groupBy("product_id", "product_name", "category") \
        .agg(
            count("*").alias("order_count"),
            sum("quantity").alias("total_quantity"),
            sum(col("price") * col("quantity")).alias("total_revenue"),
            max("timestamp").alias("last_order_time")
        ) \
        .orderBy(desc("order_count"))
    
    return top_products

def create_category_aggregation(orders_df):
    """Agregacja przychodów według kategorii"""
    if not orders_df:
        print("""
        # Przychody według kategorii
        category_revenue = orders_df \\
            .withWatermark("timestamp", "5 minutes") \\
            .groupBy("category") \\
            .agg(
                count("*").alias("total_orders"),
                sum(col("price") * col("quantity")).alias("total_revenue"),
                avg("price").alias("avg_price")
            ) \\
            .orderBy(desc("total_revenue"))
        """)
        return None
    
    category_revenue = orders_df \
        .withWatermark("timestamp", "5 minutes") \
        .groupBy("category") \
        .agg(
            count("*").alias("total_orders"),
            sum(col("price") * col("quantity")).alias("total_revenue"),
            avg("price").alias("avg_price")
        ) \
        .orderBy(desc("total_revenue"))
    
    return category_revenue

# Demonstracja agregacji na przykładowych danych
print("=== DEMONSTRACJA AGREGACJI STRUMIENIOWYCH ===")
print("\n📊 1. TOP PRODUKTY:")
create_top_products_aggregation(None)

print("\n📊 2. KATEGORIE PRODUKTÓW:")  
create_category_aggregation(None)

print("\n⏱️  KLUCZOWE CECHY AGREGACJI:")
print("- Watermark: 10 minut (tolerancja na spóźnione dane)")
print("- Grupowanie: według product_id, product_name, category")
print("- Metryki: order_count, total_quantity, total_revenue")
print("- Sortowanie: według liczby zamówień (desc)")
print("- State: zapisywany w checkpointach dla odporności")

In [ ]:
# Zaawansowane agregacje z oknem czasowym
def create_windowed_aggregations(orders_df):
    """Agregacje w oknach czasowych - trendy sprzedaży"""
    if not orders_df:
        print("🕐 OKNA CZASOWE - kod demonstracyjny:")
        print("""
        # Trendy sprzedaży w oknie 15-minutowym (aktualizacja co 5 min)
        windowed_sales = orders_df \\
            .withWatermark("timestamp", "10 minutes") \\
            .groupBy(
                window(col("timestamp"), "15 minutes", "5 minutes"),
                "category"
            ) \\
            .agg(
                count("*").alias("orders_in_window"),
                sum(col("price") * col("quantity")).alias("revenue_in_window"),
                countDistinct("product_id").alias("unique_products")
            ) \\
            .select(
                col("window.start").alias("window_start"),
                col("window.end").alias("window_end"), 
                col("category"),
                col("orders_in_window"),
                col("revenue_in_window"),
                col("unique_products")
            )
        """)
        return None
    
    # Rzeczywista implementacja
    windowed_sales = orders_df \
        .withWatermark("timestamp", "10 minutes") \
        .groupBy(
            window(col("timestamp"), "15 minutes", "5 minutes"),
            "category"
        ) \
        .agg(
            count("*").alias("orders_in_window"),
            sum(col("price") * col("quantity")).alias("revenue_in_window"),
            countDistinct("product_id").alias("unique_products")
        ) \
        .select(
            col("window.start").alias("window_start"),
            col("window.end").alias("window_end"),
            col("category"),
            col("orders_in_window"), 
            col("revenue_in_window"),
            col("unique_products")
        )
    
    return windowed_sales

# Symulacja wyników na danych przykładowych
print("=== DEMONSTRACJA OKIEN CZASOWYCH ===")
create_windowed_aggregations(None)

# Przykład wyników okien czasowych
import pandas as pd
from datetime import datetime, timedelta

base_time = datetime.now()
sample_windows = [
    {
        'window_start': base_time - timedelta(minutes=15),
        'window_end': base_time,
        'category': 'Electronics', 
        'orders_in_window': 25,
        'revenue_in_window': 3750.50,
        'unique_products': 8
    },
    {
        'window_start': base_time - timedelta(minutes=15),
        'window_end': base_time,
        'category': 'Clothing',
        'orders_in_window': 18, 
        'revenue_in_window': 1620.25,
        'unique_products': 5
    }
]

print("\n📈 Przykład wyników okien czasowych:")
df_windows = pd.DataFrame(sample_windows)
for _, row in df_windows.iterrows():
    print(f"🕐 {row['window_start'].strftime('%H:%M')}-{row['window_end'].strftime('%H:%M')} | "
          f"{row['category']}: {row['orders_in_window']} zamówień, "
          f"${row['revenue_in_window']:.2f}, {row['unique_products']} produktów")

## 4. Podstawowe ETL: czyszczenie i transformacja danych zamówień

**ETL (Extract, Transform, Load)** w kontekście streamingu:
- **Extract**: Odczyt danych z Kafki
- **Transform**: Czyszczenie, walidacja, wzbogacanie danych  
- **Load**: Zapis do sink (dashboard, baza danych, pliki)

**Typowe transformacje w e-commerce**:
- Walidacja pól obowiązkowych
- Usuwanie duplikatów
- Konwersja typów danych
- Wyliczanie metryki biznesowych
- Filtrowanie nieprawidłowych wartości

In [ ]:
# Funkcje ETL dla czyszczenia i transformacji danych
from pyspark.sql.functions import col, when, isnan, isnull, regexp_replace, lower, trim

def clean_and_validate_orders(orders_df):
    """Czyści i waliduje dane zamówień"""
    if not orders_df:
        print("🧹 FUNKCJE CZYSZCZENIA DANYCH - kod demonstracyjny:")
        print("""
        # 1. Usunięcie rekordów z brakującymi wartościami kluczowymi
        clean_orders = orders_df.filter(
            col("order_id").isNotNull() & 
            col("product_id").isNotNull() &
            col("price").isNotNull() &
            col("quantity").isNotNull()
        )
        
        # 2. Filtrowanie nieprawidłowych wartości biznesowych
        clean_orders = clean_orders.filter(
            (col("price") > 0) & 
            (col("quantity") > 0) &
            (col("quantity") <= 100)  # Maksymalnie 100 sztuk w zamówieniu
        )
        
        # 3. Czyszczenie tekstu - normalizacja kategorii
        clean_orders = clean_orders.withColumn(
            "category_clean", 
            trim(lower(col("category")))
        )
        
        # 4. Wyliczanie metryki biznesowych
        clean_orders = clean_orders.withColumn(
            "total_value", 
            col("price") * col("quantity")
        )
        """)
        return None
    
    # Rzeczywista implementacja
    clean_orders = orders_df \
        .filter(
            col("order_id").isNotNull() & 
            col("product_id").isNotNull() &
            col("price").isNotNull() &
            col("quantity").isNotNull()
        ) \
        .filter(
            (col("price") > 0) & 
            (col("quantity") > 0) &
            (col("quantity") <= 100)
        ) \
        .withColumn("category_clean", trim(lower(col("category")))) \
        .withColumn("total_value", col("price") * col("quantity"))
    
    return clean_orders

def add_business_enrichments(orders_df):
    """Dodaje wzbogacenia biznesowe"""
    if not orders_df:
        print("""
        # Wzbogacenia biznesowe
        enriched_orders = orders_df \\
            .withColumn("price_category", 
                when(col("price") < 50, "Budget")
                .when(col("price") < 200, "Mid-range") 
                .otherwise("Premium")
            ) \\
            .withColumn("order_hour", hour(col("timestamp"))) \\
            .withColumn("is_weekend", 
                when(dayofweek(col("timestamp")).isin([1, 7]), True)
                .otherwise(False)
            )
        """)
        return None
    
    enriched_orders = orders_df \
        .withColumn("price_category", 
            when(col("price") < 50, "Budget")
            .when(col("price") < 200, "Mid-range")
            .otherwise("Premium")
        ) \
        .withColumn("order_hour", hour(col("timestamp"))) \
        .withColumn("is_weekend", 
            when(dayofweek(col("timestamp")).isin([1, 7]), True)
            .otherwise(False)
        )
    
    return enriched_orders

# Demonstracja ETL na przykładowych danych
print("=== DEMONSTRACJA TRANSFORMACJI ETL ===")
print("\n🧹 1. CZYSZCZENIE I WALIDACJA:")
clean_and_validate_orders(None)

print("\n💰 2. WZBOGACENIA BIZNESOWE:")
add_business_enrichments(None)

# Przykład zastosowania na danych testowych
sample_data = [
    {"order_id": "ORD-001", "product_id": "PROD-001", "price": 299.99, "quantity": 1, "category": " Electronics ", "timestamp": "2024-01-15T14:30:00"},
    {"order_id": "ORD-002", "product_id": "PROD-002", "price": -50.0, "quantity": 2, "category": "Clothing", "timestamp": "2024-01-15T14:31:00"},  # Nieprawidłowa cena
    {"order_id": None, "product_id": "PROD-003", "price": 45.99, "quantity": 1, "category": "Books", "timestamp": "2024-01-15T14:32:00"},  # Brak order_id
    {"order_id": "ORD-004", "product_id": "PROD-001", "price": 299.99, "quantity": 150, "category": "Electronics", "timestamp": "2024-01-15T14:33:00"},  # Za duża ilość
]

print(f"\n📊 Przykład przed czyszczeniem: {len(sample_data)} rekordów")
print("- 1 prawidłowy rekord")  
print("- 1 z nieprawidłową ceną (-50.0)")
print("- 1 z brakującym order_id")
print("- 1 z za dużą ilością (150)")
print("\n✅ Po czyszczeniu: zostanie 1 prawidłowy rekord")
print("✨ Po wzbogaceniu: +price_category, +order_hour, +is_weekend")

## 5. Wizualizacja wyników w dashboardzie (opcjonalnie)

**System ASEED** zawiera kompleksowy dashboard z:
- **Real-time metryki**: łączne zamówienia, przychody, zamówienia/minutę
- **Wykresy interaktywne**: top produkty (Bar Chart), kategorie (Doughnut Chart)  
- **Live data**: najnowsze zamówienia w czasie rzeczywistym
- **WebSocket połączenie**: natychmiastowe aktualizacje z Spark

**Technologie**:
- **Backend**: Flask + SocketIO + REST API
- **Frontend**: Bootstrap + Chart.js + JavaScript
- **Integracja**: Spark wysyła dane przez HTTP POST do dashboard

In [ ]:
# Integracja z dashboardem ASEED
import requests
import matplotlib.pyplot as plt
import seaborn as sns

def send_to_dashboard(endpoint, data, dashboard_url="http://localhost:5005"):
    """Wysyła dane do dashboard ASEED"""
    try:
        response = requests.post(f"{dashboard_url}/api/{endpoint}", json=data, timeout=5)
        if response.status_code == 200:
            print(f"✅ Wysłano dane do dashboard: {endpoint}")
            return True
        else:
            print(f"⚠️  Błąd dashboard ({response.status_code}): {response.text}")
            return False
    except Exception as e:
        print(f"❌ Nie można połączyć z dashboard: {e}")
        return False

# Symulacja danych dashboard
def simulate_dashboard_data():
    """Symuluje dane które byłyby wysłane do dashboard"""
    
    # Top produkty
    top_products = [
        {"product_name": "Smart Watch Premium", "order_count": 45, "category": "Electronics"},
        {"product_name": "Fashion Jacket", "order_count": 32, "category": "Clothing"}, 
        {"product_name": "Coffee Machine Pro", "order_count": 28, "category": "Home"},
        {"product_name": "Running Shoes", "order_count": 25, "category": "Sports"},
        {"product_name": "Python Book", "order_count": 18, "category": "Books"}
    ]
    
    # Kategorie
    categories = [
        {"category": "Electronics", "total_revenue": 8750.50, "order_count": 45},
        {"category": "Clothing", "total_revenue": 3200.25, "order_count": 32},
        {"category": "Home", "total_revenue": 5600.75, "order_count": 28},
        {"category": "Sports", "total_revenue": 3250.00, "order_count": 25}, 
        {"category": "Books", "total_revenue": 810.50, "order_count": 18}
    ]
    
    # Metryki ogólne
    metrics = {
        "total_orders": 148,
        "total_revenue": 21611.00,
        "orders_per_minute": 12
    }
    
    return top_products, categories, metrics

# Demonstracja z wizualizacją
print("=== DEMONSTRACJA DASHBOARD ASEED ===")

# Pobierz przykładowe dane
top_products, categories, metrics = simulate_dashboard_data()

print(f"\n📊 METRYKI OGÓLNE:")
print(f"- Łączne zamówienia: {metrics['total_orders']}")
print(f"- Łączny przychód: ${metrics['total_revenue']:,.2f}")
print(f"- Zamówienia/minutę: {metrics['orders_per_minute']}")

print(f"\n🏆 TOP 3 PRODUKTY:")
for i, product in enumerate(top_products[:3], 1):
    print(f"{i}. {product['product_name']}: {product['order_count']} zamówień")

print(f"\n🏷️  PRZYCHODY WEDŁUG KATEGORII:")
for category in categories:
    print(f"- {category['category']}: ${category['total_revenue']:,.2f}")

# Wizualizacja danych (podobnie jak w dashboard)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(15, 6))

# Top produkty - Bar Chart
products_names = [p['product_name'][:15] for p in top_products]
products_counts = [p['order_count'] for p in top_products]

ax1.bar(products_names, products_counts, color='skyblue', alpha=0.8)
ax1.set_title('Top Produkty (liczba zamówień)', fontsize=14, fontweight='bold')
ax1.set_xlabel('Produkt')
ax1.set_ylabel('Liczba zamówień')
ax1.tick_params(axis='x', rotation=45)

# Kategorie - Pie Chart
categories_names = [c['category'] for c in categories]
categories_revenue = [c['total_revenue'] for c in categories]

ax2.pie(categories_revenue, labels=categories_names, autopct='%1.1f%%', startangle=90)
ax2.set_title('Przychody według kategorii', fontsize=14, fontweight='bold')

plt.tight_layout()
plt.show()

print("\n💡 Aby uruchomić rzeczywisty dashboard:")
print("   python3 aseed.py start")
print("   Otwórz: http://localhost:5005")

## 6. Testowanie i walidacja pipeline'u streamingowego

**Testy w systemach streamingowych** wymagają specjalnego podejścia:
- **Unit testy**: pojedyncze funkcje transformacji
- **Integration testy**: end-to-end pipeline  
- **Performance testy**: przepustowość i latencja
- **Data quality testy**: poprawność agregacji

**Narzędzia testowe**:
- **pytest**: framework testowy dla Python
- **Spark Testing Base**: narzędzia do testowania Spark
- **Testcontainers**: izolowane środowiska testowe (Kafka)

In [ ]:
# Przykłady testów dla pipeline ASEED
import unittest
from datetime import datetime

class TestASEEDPipeline(unittest.TestCase):
    """Testy jednostkowe dla funkcji ETL"""
    
    def test_order_validation(self):
        """Test walidacji zamówień"""
        # Przykładowe dane testowe
        valid_order = {
            "order_id": "ORD-123", 
            "product_id": "PROD-001",
            "price": 99.99,
            "quantity": 2
        }
        
        invalid_orders = [
            {"order_id": None, "product_id": "PROD-001", "price": 99.99, "quantity": 2},  # Brak ID
            {"order_id": "ORD-124", "product_id": "PROD-001", "price": -10, "quantity": 2},  # Ujemna cena
            {"order_id": "ORD-125", "product_id": "PROD-001", "price": 99.99, "quantity": 0},  # Zero ilość
        ]
        
        print("🧪 Test walidacji zamówień:")
        print(f"✅ Prawidłowe zamówienie: {valid_order['order_id']}")
        
        for i, order in enumerate(invalid_orders, 1):
            reason = "brak order_id" if not order.get("order_id") else \
                    "ujemna cena" if order.get("price", 0) < 0 else \
                    "zero ilość"
            print(f"❌ Nieprawidłowe zamówienie {i}: {reason}")
        
        # W rzeczywistym teście:
        # self.assertTrue(validate_order(valid_order))
        # for invalid_order in invalid_orders:
        #     self.assertFalse(validate_order(invalid_order))
    
    def test_aggregation_logic(self):
        """Test logiki agregacji"""
        # Przykładowe zamówienia dla tego samego produktu
        orders = [
            {"product_id": "PROD-001", "quantity": 2, "price": 100},
            {"product_id": "PROD-001", "quantity": 1, "price": 100},
            {"product_id": "PROD-002", "quantity": 3, "price": 50}
        ]
        
        # Oczekiwane wyniki agregacji
        expected_results = {
            "PROD-001": {"order_count": 2, "total_quantity": 3, "total_revenue": 300},
            "PROD-002": {"order_count": 1, "total_quantity": 3, "total_revenue": 150}
        }
        
        print(f"\n🧪 Test agregacji:")
        print(f"📝 Dane wejściowe: {len(orders)} zamówień")
        
        for product_id, expected in expected_results.items():
            print(f"✅ {product_id}: {expected['order_count']} zamówień, "
                  f"{expected['total_quantity']} sztuk, ${expected['total_revenue']} przychód")
        
        # W rzeczywistym teście:
        # actual_results = aggregate_orders(orders)
        # self.assertEqual(actual_results, expected_results)

def test_data_quality():
    """Test jakości danych - sprawdzenie poprawności wyników"""
    
    # Symulacja danych z systemu
    system_metrics = {
        "total_orders": 150,
        "total_revenue": 12750.50,
        "unique_products": 15,
        "avg_order_value": 85.00
    }
    
    print("\n🔍 Test jakości danych:")
    
    # Test 1: Spójność metryk
    calculated_avg = system_metrics["total_revenue"] / system_metrics["total_orders"]
    avg_diff = abs(calculated_avg - system_metrics["avg_order_value"])
    
    if avg_diff < 0.01:  # Tolerancja na błędy zaokrąglenia
        print("✅ Średnia wartość zamówienia jest spójna")
    else:
        print(f"❌ Niespójność w średniej wartości: {avg_diff:.2f}")
    
    # Test 2: Logiczne granice
    checks = [
        ("total_orders", system_metrics["total_orders"] > 0, "Liczba zamówień > 0"),  
        ("total_revenue", system_metrics["total_revenue"] > 0, "Przychód > 0"),
        ("unique_products", system_metrics["unique_products"] <= system_metrics["total_orders"], 
         "Produkty <= zamówienia"),
        ("avg_order_value", 0 < system_metrics["avg_order_value"] < 10000, 
         "Średnia wartość w rozsądnych granicach")
    ]
    
    for field, condition, description in checks:
        if condition:
            print(f"✅ {description}")
        else:
            print(f"❌ {description} - FAILED")

def run_performance_test():
    """Symulacja testu wydajności"""
    print("\n⚡ Test wydajności pipeline:")
    
    # Symulacja metryk wydajności
    performance_metrics = {
        "throughput_orders_per_sec": 50,
        "avg_processing_latency_ms": 100,
        "memory_usage_mb": 256,
        "cpu_usage_percent": 45
    }
    
    # Progi wydajności
    thresholds = {
        "min_throughput": 30,
        "max_latency_ms": 500, 
        "max_memory_mb": 512,
        "max_cpu_percent": 80
    }
    
    for metric, value in performance_metrics.items():
        threshold_key = None
        if "throughput" in metric:
            threshold_key = "min_throughput"
            passed = value >= thresholds[threshold_key]
        elif "latency" in metric:
            threshold_key = "max_latency_ms"
            passed = value <= thresholds[threshold_key]
        elif "memory" in metric:
            threshold_key = "max_memory_mb"
            passed = value <= thresholds[threshold_key]
        elif "cpu" in metric:
            threshold_key = "max_cpu_percent"
            passed = value <= thresholds[threshold_key]
        
        status = "✅" if passed else "❌"
        print(f"{status} {metric}: {value} (próg: {thresholds[threshold_key]})")

# Uruchomienie testów demonstracyjnych
print("=== DEMONSTRACJA TESTÓW ASEED PIPELINE ===")

# Testy jednostkowe
test_suite = TestASEEDPipeline()
test_suite.test_order_validation()
test_suite.test_aggregation_logic()

# Testy jakości danych
test_data_quality()

# Test wydajności  
run_performance_test()

print("\n💡 Aby uruchomić pełne testy:")
print("   cd /home/natan/ASEED")
print("   python -m pytest tests/ -v")
print("   python aseed.py test --minutes 2 --rate 30")

## Podsumowanie i wnioski akademickie

### ✅ Spełnienie wymagań projektu

**Zgodność z poleceniem**:
- ✅ **Kafka topic**: zamówienia z wymaganymi polami (order_id, product_id, price, timestamp)
- ✅ **Spark Structured Streaming**: analiza top produktów w czasie rzeczywistym
- ✅ **Streaming aggregations**: grupowanie, liczenie, sumowanie w oknach czasowych
- ✅ **ETL patterns**: extract (Kafka) → transform (czyszczenie) → load (dashboard)

### 🎯 Cele edukacyjne osiągnięte

**1. Praktyka streaming aggregations**:
- Agregacje grupowe (`groupBy`, `agg`)
- Funkcje okna czasowego (`window`, `watermark`) 
- Stateful operations z checkpointingiem

**2. Podstawowe wzorce ETL**:
- Walidacja i czyszczenie danych w czasie rzeczywistym
- Transformacje biznesowe (wyliczanie wartości, kategoryzacja)
- Pipeline data quality z monitoringiem

**3. Architektura mikrousług**:
- Rozdzielenie odpowiedzialności (simulator, processor, dashboard)
- Asynchroniczna komunikacja przez Kafka
- REST API i WebSocket dla real-time UI

### 📊 Wartość biznesowa

**Metryki monitorowane**:
- Top produkty według liczby zamówień
- Przychody według kategorii produktów  
- Trendy sprzedaży w oknach czasowych
- Wskaźniki wydajności (zamówienia/minutę)

**Przypadki użycia**:
- Wykrywanie trendów sprzedażowych w czasie rzeczywistym
- Optymalizacja zarządzania zapasami
- Personalizacja rekomendacji produktów
- Monitoring wydajności systemu e-commerce

### 🔧 Aspekty techniczne

**Zalety architektury**:
- **Skalowalność**: Kafka partitioning + Spark parallelization
- **Odporność**: Checkpointing + fault tolerance
- **Elastyczność**: Modularna architektura z wymienialnymi komponentami
- **Monitoring**: Logi, metryki, dashboard w czasie rzeczywistym

**Możliwe rozszerzenia**:
- Machine Learning (predykcja sprzedaży, anomalie)
- Większa liczba sink'ów (bazy danych, analytics platforms)
- Complex event processing (wzorce zachowań klientów)
- Auto-scaling na podstawie load'u

---

### 💡 Uruchomienie systemu ASEED

```bash
# Sklonuj i zainstaluj
git clone https://github.com/NatanTulo/ASEED.git
cd ASEED
./install.sh

# Uruchom wszystkie komponenty
python3 aseed.py start

# Otwórz dashboard
http://localhost:5005

# Test z danymi
python3 aseed.py test --minutes 5 --rate 20
```

**System gotowy do prezentacji i oceny! 🎉**